In [1]:
import os
import csv
import json
from tqdm import tqdm
import gc
import numpy as np
import torch

from transformers import AutoModel, DPRQuestionEncoder
from datasets import load_dataset
from beir.datasets.data_loader import GenericDataLoader
from beir import util
from pyserini.search.lucene import LuceneSearcher
from torchviz import make_dot
from losses import contrastive_loss, InBatchNegativeLoss
from model_longtriever import Longtriever

from model_biencoder import BiEncoder, LongBiEncoder
from preprocessing.preprocess_utils import get_triplets_dataloader, get_pairs_dataloader

import dotenv
dotenv.load_dotenv()

os.environ["JAVA_HOME"] = "/usr/lib64/openjdk-21"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

/Tmp/lvpoellhuber/bmir-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'contrastive_loss' from 'losses' (/u/poellhul/Documents/Masters/benchmarkIR-slurm/src/retrieval/losses.py)

# Parameter Comparisons

## BiEncoder
Comparing E5's automatic implementation and mine's. 

In [ ]:

custom_model = BiEncoder(model_path = ("intfloat/e5-base-v2", "intfloat/e5-base-v2"), sep=" [SEP] ")
custom_q_model = custom_model.q_model.to("cpu")
base_model = AutoModel.from_pretrained("intfloat/e5-base-v2").to("cpu")

# Compare weights
custom_weights = custom_q_model.state_dict().keys()

for name, param in base_model.named_parameters():
    #print(name)
    if name in custom_weights:
        custom_param = custom_q_model.state_dict()[name]
        if torch.equal(param, custom_param):
            #print(f"Layer {name} matches")
            pass
        else:
            print(f"Layer {name} does not match")

    else:
        print(f"Layer {name} not found.")

Comparing DPR's implementation with mine. 

In [ ]:

custom_model = BiEncoder(model_path = ("facebook/dpr-question_encoder-single-nq-base", "facebook/dpr-ctx_encoder-single-nq-base"), sep=" [SEP] ")
custom_q_model = custom_model.q_model.to("cpu")
base_model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to("cpu")

# Compare weights
custom_weights = custom_q_model.state_dict().keys()

for name, param in base_model.named_parameters():
    #print(name)
    if name in custom_weights:
        custom_param = custom_q_model.state_dict()[name]
        if torch.equal(param, custom_param):
            #print(f"Layer {name} matches")
            pass
        else:
            print(f"Layer {name} does not match")

    else:
        print(f"Layer {name} not found.")

## Longtriever
Why the hell does longtriever NOT WANNA INITIALIZE BERT PROPERlY??

In [2]:
longtriever = Longtriever.from_pretrained(
            "google-bert/bert-base-uncased",
            torch_dtype="auto",
            trust_remote_code=True,
            attn_implementation="eager",
            cache_dir=None
    ).to("cpu")
base_model = AutoModel.from_pretrained("google-bert/bert-base-uncased").to("cpu")

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
You are using a model of type bert to instantiate a model of type longtriever. This is not supported for all configurations of models and can yield errors.
Some weights of Longtriever were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['doc_embeddings', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.information_exchanging_layer.0.attention.output.LayerNorm.bias', 'encoder.information_exchanging_layer.0.attention.output.LayerNorm.weight', 'encoder.information_exchanging_layer.0.attention.output.dense.bias', 'encoder.information_exchanging_layer.0.attention.output.dense.weight', 'encoder.information_exchanging_layer.0.attention.self.key.bias', 'encoder.information_exchanging_layer.0.attenti

In [8]:

# Get the state_dict of the BERT encoder
bert_state_dict = base_model.state_dict()

# Get the state_dict of the Longtriever model
longtriever_state_dict = longtriever.state_dict()

# Map weights from BERT to Longtriever
new_state_dict = {}
for bert_key, bert_value in bert_state_dict.items():
    # Replace layer names to match Longtriever's naming convention
    if "layer" in bert_key:
        # Example: Map BERT's "layer.X" to Longtriever's "text_encoding_layers.X"
        new_key = bert_key.replace("layer", "text_encoding_layer")
        if new_key in longtriever_state_dict:
            new_state_dict[new_key] = bert_value

        # Example: Map BERT's "layer.X" to Longtriever's "information_exchanging.X"
        new_key = bert_key.replace("layer", "information_exchanging_layer")
        if new_key in longtriever_state_dict:
            new_state_dict[new_key] = bert_value
    else:
        if bert_key in longtriever_state_dict:
            new_state_dict[bert_key] = bert_value

# Update Longtriever's state_dict with the new weights
longtriever_state_dict.update(new_state_dict)

# Load the updated state_dict into Longtriever
longtriever.load_state_dict(longtriever_state_dict)

<All keys matched successfully>

In [9]:
longtriever.modules

<bound method Module.modules of Longtriever(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BlockLevelContextawareEncoder(
    (text_encoding_layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [10]:
base_model.modules

<bound method Module.modules of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropou

In [11]:
for name, param in base_model.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [12]:
lt_weights = longtriever.state_dict().keys()

for name, param in base_model.named_parameters():
    text_name = name.replace("layer", "text_encoding_layer")
    # print(name)
    if text_name in lt_weights:
        lt_text_param = longtriever.state_dict()[text_name]
        if torch.equal(param, lt_text_param):
            print(f"Layer {text_name} matches")
            pass
        else:
            print(f"Layer {text_name} does not match")
            pass
    else:
        print(f"Layer {text_name} not found.")
        pass

    info_name = name.replace("layer", "information_exchanging_layer")
    if info_name in lt_weights:
        lt_info_param = longtriever.state_dict()[info_name]
        if torch.equal(param, lt_info_param):
            print(f"Layer {info_name} matches")
            pass
        else:
            print(f"Layer {info_name} does not match")
            pass
    else:
        print(f"Layer {info_name} not found.")
        pass


Layer embeddings.word_embeddings.weight matches
Layer embeddings.word_embeddings.weight matches
Layer embeddings.position_embeddings.weight matches
Layer embeddings.position_embeddings.weight matches
Layer embeddings.token_type_embeddings.weight matches
Layer embeddings.token_type_embeddings.weight matches
Layer embeddings.LayerNorm.weight matches
Layer embeddings.LayerNorm.weight matches
Layer embeddings.LayerNorm.bias matches
Layer embeddings.LayerNorm.bias matches
Layer encoder.text_encoding_layer.0.attention.self.query.weight matches
Layer encoder.information_exchanging_layer.0.attention.self.query.weight matches
Layer encoder.text_encoding_layer.0.attention.self.query.bias matches
Layer encoder.information_exchanging_layer.0.attention.self.query.bias matches
Layer encoder.text_encoding_layer.0.attention.self.key.weight matches
Layer encoder.information_exchanging_layer.0.attention.self.key.weight matches
Layer encoder.text_encoding_layer.0.attention.self.key.bias matches
Layer enc

# Torchviz

In [ ]:
batch_size = 20

In [3]:
dpr_model = BiEncoder(
    model_path=("google-bert/bert-base-uncased", "google-bert/bert-base-uncased"),
    normalize=False,
    prompts={"query": "", "passage": ""},
    attn_implementation="eager", 
    sep = " [SEP] ", 
    batch_size=batch_size
)
dpr_model.train()

In [4]:
dataset_path = os.path.join("/Tmp/lvpoellhuber/datasets/nq", "train_triplets.pt")
dataloader = get_triplets_dataloader(batch_size=batch_size, dataset_path=dataset_path,)

In [ ]:
dataset_path = os.path.join("/Tmp/lvpoellhuber/datasets/nq", "train_pairs.pt")
dataloader = get_pairs_dataloader(batch_size=batch_size, dataset_path=dataset_path,)

In [5]:
dpr_params = {}

q_params = dict(dpr_model.q_model.named_parameters())
doc_params = dict(dpr_model.doc_model.named_parameters())
for param in q_params.keys():
    q_param = q_params[param]
    doc_param = doc_params[param]

    dpr_params["q_encoder."+param] = q_param 
    dpr_params["doc_encoder."+param] = doc_param 

In [6]:
batch = [item for item in dataloader][0]

queries = batch["queries"]
# documents = batch["documents"]
positives = batch["positives"]
negatives = batch["negatives"]
documents = positives + negatives

In [7]:
def dpr_step(queries, documents):
    q_embeddings = dpr_model.encode_queries(queries, convert_to_tensor=True) # All three 16x512
    doc_embeddings = dpr_model.encode_corpus(documents, convert_to_tensor=True) # All three 16x512

    loss = contrastive_loss(q_embeddings, doc_embeddings)

    return loss

# loss = dpr_step(queries, documents)

In [8]:
make_dot(dpr_step(queries, documents), params=dpr_params, show_attrs=True).render("/u/poellhul/Documents/Masters/benchmarkIR-slurm/src/retrieval/dual_graph")

'/u/poellhul/Documents/Masters/benchmarkIR-slurm/src/retrieval/dual_graph.pdf'

In [9]:
make_dot(dpr_model.encode_corpus(documents, convert_to_tensor=True), params=dpr_params, show_attrs=True).render("/u/poellhul/Documents/Masters/benchmarkIR-slurm/src/retrieval/doc_graph")

'/u/poellhul/Documents/Masters/benchmarkIR-slurm/src/retrieval/doc_graph.pdf'

### Longtriever

In [3]:
batch_size = 3

biencoder = LongBiEncoder(
    model_path="/Tmp/lvpoellhuber/models/longtriever/pretrained/bert-base-uncased",
    normalize=False,
    prompts={"query": "", "passage": ""},
    attn_implementation="eager", 
    sep = " [SEP] ", 
    batch_size=batch_size,
    max_block_length=512, 
    max_num_blocks=8,
    model_type =  "longtriever"
)
biencoder.train()

In [4]:
dataloader = get_pairs_dataloader(batch_size=batch_size, dataset_path="/Tmp/lvpoellhuber/datasets/msmarco-doc/train_pairs.pt")

biencoder_params = {}

q_params = dict(biencoder.q_model.named_parameters())
# doc_params = dict(biencoder.doc_model.named_parameters())
for param in q_params.keys():
    q_param = q_params[param]
    # doc_param = doc_params[param]

    biencoder_params["q_encoder."+param] = q_param 
    # biencoder_params["doc_encoder."+param] = doc_param 

In [5]:
loss_fct = InBatchNegativeLoss()
def biencoder_step(queries, documents):
    q_embeddings = biencoder.encode_queries(queries, convert_to_tensor=True) # All three 16x512
    doc_embeddings = biencoder.encode_corpus(documents, convert_to_tensor=True) # All three 16x512

    loss = loss_fct(q_embeddings, doc_embeddings)

    return loss

# loss = dpr_step(queries, documents)

In [6]:
batch = [item for item in dataloader][0]

queries = batch["queries"]
documents = batch["documents"]

In [7]:
make_dot(biencoder_step(queries, documents), params=biencoder_params, show_attrs=True).render("/u/poellhul/Documents/Masters/benchmarkIR-slurm/src/retrieval/single_longtriever")

'/u/poellhul/Documents/Masters/benchmarkIR-slurm/src/retrieval/single_longtriever.pdf'

# Dataloader inspection

## NQ

In [ ]:
dataset_path = os.path.join("/Tmp/lvpoellhuber/datasets/nq/train_pairs.pt")
dataloader = get_pairs_dataloader(batch_size=12, dataset_path=dataset_path)
       

In [ ]:
for batch in dataloader:
    print("huh")

## MS Marco Doc

In [2]:
import numpy as np
import random
def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)


### Pairs

In [3]:
pairs_dataloader = get_pairs_dataloader(
        batch_size=3, 
        dataset_path="/Tmp/lvpoellhuber/datasets/msmarco-doc/train_pairs.pt", 
        pin_memory=True, 
        prefetch_factor=2, 
        num_workers = 4
    )

In [4]:
random_batch = random.randint(0, len(pairs_dataloader)-1)
for i, batch in enumerate(pairs_dataloader):
    queries = batch["queries"]
    documents = batch["documents"]

    print("huh")
    break

huh


In [5]:
queries

['what is the futa tax',
 'what county valdosta ga',
 'how many mg of caffeine in coffee']

In [6]:
documents

[{'_id': 'D944658',
  'title': 'Federal Unemployment Tax Act',
  'text': '"From Wikipedia, the free encyclopedianavigation search This article is part of a series on Taxation in the United States of America Federal taxation [show]State and local taxation [show]Federal tax reform [show]United States portalv t e The Federal Unemployment Tax Act (or FUTA, I. R. C. ch. 23) is a United States federal law that imposes a federal employer tax used to help fund state workforce agencies. Employers report this tax by filing an annual Form 940 with the Internal Revenue Service. In some cases, the employer is required to pay the tax in installments during the tax year. FUTA covers a federal share of the costs of administering the unemployment insurance (UI) and job service programs in every state. In addition, FUTA pays one-half of the cost of extended unemployment benefits (during periods of high unemployment) and provides for a fund from which states may borrow, if necessary, to pay benefits. Con

### Triplets

In [28]:
triplets_dataloader = get_triplets_dataloader(
        batch_size=3, 
        dataset_path="/Tmp/lvpoellhuber/datasets/msmarco-doc/train_triplets.pt", 
        pin_memory=True, 
        prefetch_factor=2, 
        num_workers = 4
    )

In [35]:
random_batch = random.randint(0, len(triplets_dataloader)-1)
for i, batch in enumerate(triplets_dataloader):
    queries = batch["queries"]
    documents = batch["documents"]

    print("huh")
    break

huh


In [36]:
queries

['schlage customer service phone number']

In [ ]:
documents

[{'text': 'What Is Schlage Locks Customer Service Phone Number? Asked in Customer Service Numbers Find the Schlage Locks phone number for customer service at 888-805-9837 helpline to contact Locks for technical support online help to fix your problem and get the issue solved. What Is The Use Of Car Insurance Login Account By MAPFRE Commerce New 4:07 hours ago (A) by Whitmire Olmos Other Relevant Answers on This Topic? Contact ushttp://www.schlage.com/en/home/contact-us.html Entry Door Knobs, Door Locks & Hardware | Schlage. Press Enter .... Top Asked Questions. Customer Care | Service phone number for Residential Schlage. Contact Ushttp://us.allegion.com/irst/pages/contact_us.aspx Schlage Multi Point Lock · Von Duprin Security Indicators · Environmental .... Primary Phone #:*. Address:* ... Schlage Electronic Von Duprin ... email phone. I would like to receive future communications from Allegion, Yes No ... REGISTERED IN IRELAND WITH LIMITED LIABILITY REGISTERED NUMBER 527370. Allegion

: 

## MS Marco Passage
### Pairs

In [20]:
pairs_dataloader = get_pairs_dataloader(
        batch_size=3, 
        dataset_path="/Tmp/lvpoellhuber/datasets/msmarco-passage/train_pairs.pt", 
        pin_memory=True, 
        prefetch_factor=2, 
        num_workers = 4
    )

In [21]:
random_batch = random.randint(0, len(pairs_dataloader)-1)
for i, batch in enumerate(pairs_dataloader):
    queries = batch["queries"]
    documents = batch["documents"]

    print("huh")
    break

huh


In [22]:
queries

[{'_id': '864691', 'text': 'what is ytterbium', 'metadata': {}},
 {'_id': '246676',
  'text': 'how long do sperm cells mature for',
  'metadata': {}},
 {'_id': '1066177', 'text': 'why do you get red eye', 'metadata': {}}]

In [23]:
documents

[{'_id': '4784656',
  'text': 'References. Ytterbium (IPA : /ÉªË\x88tÉ\x9bË\x90(r)biÉ\x99m/) is a chemical element in the periodic table that has the symbol Yb and atomic number 70. A soft silvery metallic element, ytterbium is a rare earth of the lanthanide series and is found in the minerals gadolinite, monazite, and xenotime. The element is sometimes associated with yttrium or other related elements and is used in certain steels.'},
 {'_id': '2406154',
  'text': 'Hormones associated with the menstrual cycle (period) cause eggs inside the ovaries to mature. Every 28 days or so, one mature egg is released from the ovary. This is called ovulation. After the egg is released, it moves into the fallopian tube where it stays for about 24 hours. If the egg is not fertilized during that time, the egg disintegrates (breaks down) and menstruation (your period) begins 2 weeks later.'},
 {'_id': '4848480',
  'text': "We've all experienced red, bloodshot eyes from time to time, affecting one or b

### Triplets

In [24]:
triplets_dataloader = get_triplets_dataloader(
        batch_size=3, 
        dataset_path="/Tmp/lvpoellhuber/datasets/msmarco-passage/train_triplets.pt", 
        pin_memory=True, 
        prefetch_factor=2, 
        num_workers = 4
    )

In [25]:
random_batch = random.randint(0, len(triplets_dataloader)-1)
for i, batch in enumerate(triplets_dataloader):
    queries = batch["queries"]
    documents = batch["documents"]

    print("huh")
    break

huh


In [26]:
queries

['what does forskolin do?']

In [27]:
documents

[{'text': "Forskolin has leapt from the lab to the gym, and with good reason! It could help you burn fat, support healthy testosterone levels, and get more results out of your training. Here's why you should give it a try!",
  'title': None},
 {'text': 'Here are the factors that need to be present in a Forskolin extract brand so it can be considered a top quality Forskolin supplement: 1. Must Contain 20% Forskolin Extract. 20% pure and potent Forskolin extract is the golden standard in the industry. A lot of brands in the market only contain 10% or 15% and sadly, no matter what promises that brand make, it wonâ\x80\x99t work for weight loss. If a product does not contain 20% Forskolin, then forget about it. 2.',
  'title': None}]

# Data Inspection

In [3]:
data_path = "/Tmp/lvpoellhuber/datasets/msmarco-doc-short"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

100%|██████████| 256146/256146 [00:03<00:00, 78597.46it/s]


In [4]:
corpus["D1810099"]

{'text': "Joint Publication 1-02Department of Defense Dictionary of Military and Associated Terms8 November 2010 (As Amended Through 15 April 2013) PREFACE1. Scope The Joint Publication 1-02, Department of Defense Dictionary of Military and Associated Terms sets forth standard US military and associated terminology to encompass the joint activity of the Armed Forces of the United States. These military and associated terms, together with their definitions, constitute approved Department of Defense (DOD) terminology for general use by all DOD components.2. Purpose This publication supplements standard English-language dictionaries and standardizes military and associated terminology to improve communication and mutual understanding within DOD, with other federal agencies, and among the United States and its allies.3. Application This publication applies to the Office of the Secretary of Defense, the Services, the Joint Staff, combatant commands, DOD agencies, and all other DOD component

In [5]:
queries["610190"]

'what county is new castle,pa in'

# Hidden State Shapes

In [311]:
hidden_states= torch.rand(3, 8, 513, 768)

hidden_states[:, :, 0, :] = -1

hidden_states[0, 0, :, :]


tensor([[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [ 0.1767,  0.7940,  0.0560,  ...,  0.0948,  0.9265,  0.5313],
        [ 0.2671,  0.3312,  0.9351,  ...,  0.9478,  0.0695,  0.5010],
        ...,
        [ 0.5979,  0.3564,  0.4494,  ...,  0.1155,  0.8178,  0.7187],
        [ 0.0975,  0.5869,  0.5117,  ...,  0.7252,  0.8423,  0.8690],
        [ 0.6025,  0.1467,  0.8125,  ...,  0.7794,  0.3533,  0.2911]])

Right above is the hidden state for block 0 of example 0. Its first token is the DOC token, which is assigned the value -1.0000. The matrix above is of shape 513x768. The **VERTICAL** line represents the words (513), while the **horizontal** line represents the vector representations (768).

The DOC token (1) will have the value -1 for its entire representation, as it represents the indexes we want to conserve. 

In [312]:
hidden_states[:, torch.arange(8), torch.arange(8)+1, :] = ((torch.arange(8)+1)*10.0).view(8, 1).repeat(3, 1, 768)


hidden_states[0, 0, :, :].round()

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [10., 10., 10.,  ..., 10., 10., 10.],
        [ 0.,  0.,  1.,  ...,  1.,  0.,  1.],
        ...,
        [ 1.,  0.,  0.,  ...,  0.,  1.,  1.],
        [ 0.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  0.,  1.,  ...,  1.,  0.,  0.]])

Now we can see the hidden state above with the CLS tokens. This is again block 0 of example 0. The CLS token is at position 1 for this block (it would be 0 without the DOC token). Each CLS token will have its own index value, multiplied by then when it is the current block's token. 

In [313]:
hidden_states[0, 1, :, :]

tensor([[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [ 0.9359,  0.4442,  0.4246,  ...,  0.7920,  0.0916,  0.8704],
        [20.0000, 20.0000, 20.0000,  ..., 20.0000, 20.0000, 20.0000],
        ...,
        [ 0.5618,  0.3202,  0.8232,  ...,  0.7665,  0.3409,  0.5628],
        [ 0.6446,  0.5704,  0.3970,  ...,  0.7285,  0.4527,  0.9810],
        [ 0.8799,  0.8465,  0.5363,  ...,  0.4101,  0.4948,  0.6691]])

We can verify with block 1 of example 0, where the CLS token (20.0000) is at position 2. 


In [314]:
cls_tokens = hidden_states[:, torch.arange(8), torch.arange(8)+1, :]/10 # Divide by ten to indicate they're not the main tokens
cls_tokens

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [2., 2., 2.,  ..., 2., 2., 2.],
         [3., 3., 3.,  ..., 3., 3., 3.],
         ...,
         [6., 6., 6.,  ..., 6., 6., 6.],
         [7., 7., 7.,  ..., 7., 7., 7.],
         [8., 8., 8.,  ..., 8., 8., 8.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [2., 2., 2.,  ..., 2., 2., 2.],
         [3., 3., 3.,  ..., 3., 3., 3.],
         ...,
         [6., 6., 6.,  ..., 6., 6., 6.],
         [7., 7., 7.,  ..., 7., 7., 7.],
         [8., 8., 8.,  ..., 8., 8., 8.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [2., 2., 2.,  ..., 2., 2., 2.],
         [3., 3., 3.,  ..., 3., 3., 3.],
         ...,
         [6., 6., 6.,  ..., 6., 6., 6.],
         [7., 7., 7.,  ..., 7., 7., 7.],
         [8., 8., 8.,  ..., 8., 8., 8.]]])

Above we verify for example 0 that the extracted CLS tokens truly are the correct ones. 

Now, I need to figure out how to select the lower diagonal and modify it. 

## Triangular matrices
### With 3x8x513x768

Unfortunately I coded this assuming the 3x8 were separate, rather than 24. It's more logical and easier to code, but it won't fit with the model. 

In [323]:
B, N, L_, D = hidden_states.shape
B, N, L_, D

(3, 8, 513, 768)

In [ ]:
# pre
pre_indices = torch.tril_indices(N, L_)
mask = pre_indices[1] != 0
filtered_indices = pre_indices[:, mask]

extended_cls_tokens = cls_tokens.repeat(N, 1, 1, 1).view(B, N, N, D)
pre_cls_indices = torch.tril_indices(N, N, offset=-1)

hidden_states[:, filtered_indices[0], filtered_indices[1], :] = extended_cls_tokens[:, pre_cls_indices[0], pre_cls_indices[1], :]

# post
post_indices = torch.triu_indices(N, L_, offset=L_-N+1)
mask = post_indices[1] != hidden_states.shape[1] - 1
filtered_indices = post_indices[:, mask]

post_cls_indices = torch.triu_indices(N, N, offset=1)

hidden_states[:, filtered_indices[0], filtered_indices[1], :] = extended_cls_tokens[:, post_cls_indices[0], post_cls_indices[1], :]


# Evaluation Memory Usage

Initial CPU memory usage (with loaded modules): ~2.7GB / 62.6GB

In [2]:
dpr_model = LongBiEncoder.from_pretrained("/Tmp/lvpoellhuber/models/longtriever/longtriever_shared")

After loading model: 2.7GB

In [ ]:
corpus, queries, qrels = GenericDataLoader(data_folder="/Tmp/lvpoellhuber/datasets/msmarco-doc").load(split="test")

Short dataset: 5.84GB
Long dataset: 

In [5]:
corpus_embeddings = np.ones((3201821, 768))

After loading dummy embeddings: 59.9GB

In [6]:
import faiss

In [ ]:
passage_ids = np.arange(3201821)
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])

for start in range(0, len(passage_ids), 50000):
    index.add(corpus_embeddings[start : start + 50000])

: 